In [11]:
import pandas as pd

# Read the CSV file
median_data = pd.read_csv('median.csv')

# read the geojson file
geojson_data = pd.read_json('geo-data.geojson')

In [13]:
median_data

,District,Wage_median,Count,Wage_mean,Wage_75%,Wage_25%,Wage_iqr,Wage_std
0,北京(Not Specified),8000,114,8248,10000,6500,3500.0,2268
1,石景山区,7000,29,7983,9000,5500,3500.0,3499
2,密云区,6500,13,7423,7500,5500,2000.0,2490
3,通州区,7000,193,7356,8100,6000,2100.0,1930
4,西城区,7000,90,7236,7500,5563,1937.5,3005
5,东城区,7050,95,7026,7675,5875,1800.0,2002
6,海淀区,6500,195,6962,7500,5000,2500.0,3142
7,朝阳区,6500,334,6959,7750,5250,2500.0,2347
8,昌平区,6500,208,6935,7500,5500,2000.0,1985
9,丰台区,6500,174,6827,7700,5063,2637.5,2315


In [12]:
# add "区" to the end of the District except for the first row
for i in range(1, len(median_data)):
    median_data.loc[i, 'District'] += '区'

In [15]:
for i in range(len(geojson_data['features'])):
    # find the row in the median data that matches the district name in the geojson data, then add the count, Wage_median and Wage_mean to the geojson data
    for j in range(len(median_data)):
        if geojson_data['features'][i]['properties']['name'] == median_data.loc[j, 'District']:
            geojson_data['features'][i]['properties']['count'] = median_data.loc[j, 'Count']
            geojson_data['features'][i]['properties']['Wage_median'] = median_data.loc[j, 'Wage_median']
            geojson_data['features'][i]['properties']['Wage_mean'] = median_data.loc[j, 'Wage_mean']
            break

In [17]:
geojson_data.keys()

Index(['type', 'features'], dtype='object')

In [25]:
import numpy as np
import pandas as pd
from pandas import json_normalize

In [26]:
df_geo = pd.DataFrame.from_dict(json_normalize(geojson_data['features']), orient='columns')

In [31]:
df_geo.head()

,type,properties.adcode,properties.name,properties.center,properties.centroid,properties.childrenNum,properties.level,properties.subFeatureIndex,properties.acroutes,properties.site,properties.count,properties.Wage_median,properties.Wage_mean,geometry.type,geometry.coordinates
0,Feature,110101,东城区,"[116.418757, 39.917544]","[116.416718, 39.912934]",0,district,0,"[100000, 110000]",https://geojson.hxkj.vip,95.0,7050,7026,MultiPolygon,"[[[[116.44364, 39.872840000000004], [116.44257..."
1,Feature,110102,西城区,"[116.366794, 39.915309]","[116.36567, 39.912028]",0,district,1,"[100000, 110000]",https://geojson.hxkj.vip,90.0,7000,7236,MultiPolygon,"[[[[116.325799, 39.896789], [116.337301, 39.89..."
2,Feature,110105,朝阳区,"[116.486409, 39.921489]","[116.513687, 39.951064]",0,district,2,"[100000, 110000]",https://geojson.hxkj.vip,334.0,6500,6959,MultiPolygon,"[[[[116.595548, 40.01751], [116.60132, 40.0138..."
3,Feature,110106,丰台区,"[116.286968, 39.863642]","[116.250342, 39.835716]",0,district,3,"[100000, 110000]",https://geojson.hxkj.vip,174.0,6500,6827,MultiPolygon,"[[[[116.259089, 39.896658], [116.266199, 39.89..."
4,Feature,110107,石景山区,"[116.195445, 39.914601]","[116.176229, 39.933205]",0,district,4,"[100000, 110000]",https://geojson.hxkj.vip,29.0,7000,7983,MultiPolygon,"[[[[116.167033, 39.888752], [116.161094, 39.89..."


In [30]:
df_geo['properties.count'] = df_geo['properties.count']*2198

In [32]:
import json
ok_json = json.loads(df_geo.to_json(orient='records'))

In [33]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [34]:
geo_format = process_to_geojson(ok_json)

In [35]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)